In [1]:
#Webdriver manager wouldn't load in the next cell, this is a work around
import sys
!pip install webdriver_manager

In [2]:
# Import Splinter and BeautifulSoup & Pandas
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [3]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/kaitlin/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [4]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)
#Search for this specific tag (div) & attribute (list_text) & wait 1 sec to make sure page loads fully

True

In [5]:
#Parse info
html = browser.html
news_soup = soup(html, 'html.parser')
#Return the first thing that matches (ie on news page, the newest article)
slide_elem = news_soup.select_one('div.list_text')

In [6]:
#Scrape the title
slide_elem.find('div', class_ ='content_title')
#slide_elem holds a ton of info, find divs where the class is "content_title"

<div class="content_title">NASA's Briefcase-Size MarCO Satellite Picks Up Honors</div>

In [7]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title
#This gets rid of the code <> in the above ouput

"NASA's Briefcase-Size MarCO Satellite Picks Up Honors"

In [8]:
#Scrape the summary text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'The twin spacecraft, the first of their kind to fly into deep space, earn a Laureate from Aviation Week & Space Technology.'

### Featured Images

In [9]:
# Visit URL
url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url)

In [10]:
# Find and click the full image button (2nd button on the page)
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()


In [11]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [12]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel
#.get(src) pulls the link

'image/featured/mars3.jpg'

In [13]:
# Use the base URL to create an absolute URL
img_url = f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{img_url_rel}'
img_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg'

Create table

In [14]:
#Create DF to read the facts table html
df = pd.read_html('https://data-class-mars-facts.s3.amazonaws.com/Mars_Facts/index.html')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [15]:
#Send the DF to HTML page
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [16]:
#Shut down the automated browsers
#browser.quit()

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

In [17]:
# 1. Use browser to visit the URL 
url = 'https://data-class-mars-hemispheres.s3.amazonaws.com/Mars_Hemispheres/index.html'
browser.visit(url)

In [18]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.
#For loop to cycle through all 4 images

for i in range(4,12,2):

    #Create an empty dictionary to store info
    hemispheres = {}

    #Open the hemisphere page
    full_image_page = browser.find_by_tag('a')[i]
    full_image_page.click()

    #Get image relative path
    html = browser.html
    img_soup = soup(html, 'html.parser')
    img_url_rel = img_soup.find('a', text='Sample').get('href')
    img_url = f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{img_url_rel}'

    #Get image title
    img_title = img_soup.select_one('h2.title').get_text()
    
    #Add details to dictionary
    hemispheres = {
        "img_url": img_url, 
        "title": img_title}
    
    #Append the list with the dictionary
    hemisphere_image_urls.append(hemispheres)
    
    #Return to index page
    return_to_index = browser.back()


In [19]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'img_url': 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [20]:
# 5. Quit the browser
browser.quit()